In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
from numpy.linalg import inv
from tqdm import tqdm
from sklearn.datasets import make_regression

In [ ]:
X, y = make_regression(n_samples = 10000,
                       n_features = 2,
                       noise = 1,
                       random_state = 42)

In [ ]:
from numpy.linalg import det
class Linear_Regression:

  def __init__(self,X,y,tolerance=None,learning_rate=None,reg_param=None):
    self.X = X
    self.y = y
    self.tolerance = tolerance
    self.learning_rate = learning_rate
    self.reg_param = reg_param

  def split_dataset(self):
    X_train, X_test, y_train, y_test = train_test_split(self.X,
                                                            self.y,
                                                            train_size=0.75,
                                                            random_state=1,
                                                            shuffle=True)
    return X_train, X_test, y_train, y_test

  def normalize_train(self,X):
    mean = np.mean(X,0)
    std = np.std(X,0)
    X = (X-mean)/std
    return X, mean, std

  def normalize_test(self,X,mean,std):
    return (X-mean)/std

  def addX0(self,X):
    return np.column_stack([np.ones(X.shape[0]),X])

  def closedformSol(self,X,y):
    return np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

  def predict(self,X):
    return X.dot(self.w)

  def lossfunction(self,X,y):
    y_hat = self.predict(X)
    return y - y_hat

  def SSE(self,X,y):
    y_hat = self.predict(X)
    return ((y-y_hat)**2).sum()

  def costfunction(self,X,y):
    return self.SSE(X,y)/2 + self.reg_param*((self.closedformSol(X,y)**2).sum())

  def costDerivative(self,X,y):
    return X.T.dot(self.predict(X) - y)

  def gradientDescent(self,X,y):
    errors = []
    last_err = float('inf')
    self.w = np.ones(X.shape[1],dtype=np.float64)

    for i in tqdm(range(X.shape[0])):
      self.w = self.w - self.learning_rate * self.costDerivative(X,y)
      current_err = self.SSE(X,y)
      diff = abs(current_err - last_err)
      last_err = current_err

      if diff < self.tolerance:
        print('model converged')
        break

      errors.append(diff)

    print(self.w)

  def regularization(self,X,y):
    return self.costfunction(X,y) + self.reg_param * (self.w)

  def fit(self):
    self.split_dataset()
    X_train, X_test, y_train, y_test = self.split_dataset()
    X_train, mean, std = self.normalize_train(X_train)
    X_test = self.normalize_test(X_test,mean,std)

    self.gradientDescent(X_train,y_train)
    print('coefficients are: {}'.format(self.w))

In [ ]:
reg = Linear_Regression(X,y,tolerance=0.00001,learning_rate=0.005)

In [ ]:
reg.fit()